In [8]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
%matplotlib inline
plt.style.use('ggplot')

In [9]:
from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout

style = {'description_width': '100px'}
slider_layout = Layout(width='99%')

In [10]:
def ode_model(z, t, beta, tao, zeta, epsilon, kappa, phi, alpha4, alpha3): #Added all parameters Reordered 
    S, E, Is, Ia, R = z #Added Is, Ia terms for symptomatic and asymptomatic 
    N = S + E + Is + Ia + R #Added Is, Ia to the sum
    dSdt = -beta*S*(Is + tao*Ia)/N + zeta*E #Added I = Is + Iu and tao*Iu is the reduced transmission rate; zeta*E is the recovery rate from E to S
    dEdt = beta*S*(Is + tao*Ia)/N - (epsilon + zeta)*E #Added I and (sigma + zeta)
    dIsdt = epsilon*kappa*E - phi*Ia - alpha4*Is # Is symptomatic
    dIadt = epsilon*(1 - kappa)*E - (alpha3 + phi)*Ia # Ia asymptomatic
    dRdt = alpha4*Is + alpha3*Ia 
 
    return [dSdt, dEdt, dIsdt, dIadt, dRdt] #S, E, S, A, R

In [11]:
def ode_solver(t, initial_conditions, params):
    initE, initIs, initIa, initR, initN= initial_conditions #Added all init names
    beta, tao, zeta, epsilon, kappa, phi, alpha4, alpha3 = params #Added all parameters
    initS = initN - (initE + initIs + initIa + initR)
    res = odeint(ode_model, [initS, initE, initIs, initIa, initR], t, args=(beta, tao, zeta, epsilon, kappa, phi, alpha4, alpha3))
    return res

In [23]:
# ref: https://www.medrxiv.org/content/10.1101/2020.04.01.20049825v1.full.pdf
initN = 2930000
initE = 1
initIs = 1
initIa = 1
initR = 0
beta = 2
tao = 0.2
zeta = 0.05
epsilon = 0.95
kappa = 0.8
phi = 0.7
alpha4 = 0.12
alpha3 = 0.2
days = 200

In [24]:
def main(initE, initIs, initIa, initR, initN, beta, tao, zeta, epsilon, kappa, phi, alpha4, alpha3, days):
    initial_conditions = [initE, initIs, initIa, initR, initN]
    params = [beta, tao, zeta, epsilon, kappa, phi, alpha4, alpha3]
    tspan = np.arange(0, days, 1)
    sol = ode_solver(tspan, initial_conditions, params)
    S, E, Is, Ia, R = sol[:, 0], sol[:, 1], sol[:, 2], sol[:, 3], sol[:, 4]
    
    # Create traces
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=tspan, y=S, mode='lines+markers', name='Susceptible'))
    fig.add_trace(go.Scatter(x=tspan, y=E, mode='lines+markers', name='Exposed'))
    fig.add_trace(go.Scatter(x=tspan, y=Is, mode='lines+markers', name='Symptomatic Infected'))
    fig.add_trace(go.Scatter(x=tspan, y=Ia, mode='lines+markers', name='Asymptomatic Infected'))
    fig.add_trace(go.Scatter(x=tspan, y=R, mode='lines+markers',name='Recovered'))
    
    if days <= 30:
        step = 1
    elif days <= 90:
        step = 7
    else:
        step = 30
    
    # Edit the layout
    fig.update_layout(title='Simulation of SESAR Model',
                       xaxis_title='Day',
                       yaxis_title='Counts',
                       title_x=0.5,
                      width=900, height=600
                     )
    fig.update_xaxes(tickangle=-90, tickformat = None, tickmode='array', tickvals=np.arange(0, days + 1, step))
    if not os.path.exists("images"):
        os.mkdir("images")
    #fig.write_image("images/seird_simulation.png")
    fig.show()

In [25]:
interact(main, 
         initE=IntSlider(min=0, max=100000, step=1, value=initE, description='initE', style=style, layout=slider_layout),
         initIs=IntSlider(min=0, max=100000, step=10, value=initIs, description='initIs', style=style, layout=slider_layout),
         initIa=IntSlider(min=0, max=100000, step=10, value=initIa, description='initIa', style=style, layout=slider_layout),
         initR=IntSlider(min=0, max=100000, step=10, value=initR, description='initR', style=style, layout=slider_layout),
         initN=IntSlider(min=0, max=1380000000, step=1000, value=initN, description='initN', style=style, layout=slider_layout),
         beta=FloatSlider(min=0, max=4, step=0.01, value=beta, description='Infection rate', style=style, layout=slider_layout),
         tao=FloatSlider(min=0, max=1, step=0.01, value=tao, description='Asymptomatic Reduced Infection Rate', style=style, layout=slider_layout),
         zeta=FloatSlider(min=0, max=1, step=0.01, value=zeta, description='Recovery from Exposure rate', style=style, layout=slider_layout),
         epsilon=FloatSlider(min=0, max=1, step=0.01, value=epsilon, description='Exposure to Infection rate', style=style, layout=slider_layout), 
         kappa=FloatSlider(min=0, max=1, step=0.01, value=kappa, description='% Symptomatic Infection', style=style, layout=slider_layout),
         phi=FloatSlider(min=0, max=1, step=0.01, value=phi, description='Asymptomatic to Symptomatic rate', style=style, layout=slider_layout),
         alpha4=FloatSlider(min=0, max=1, step=0.01, value=alpha4, description='Symptomatic Recovery rate', style=style, layout=slider_layout),
         alpha3=FloatSlider(min=0, max=1, step=0.01, value=alpha3, description='Asymptomatic Recovery rate', style=style, layout=slider_layout),
         days=IntSlider(min=1, max=600, step=7, value=days, description='Days', style=style, layout=slider_layout)
        );

interactive(children=(IntSlider(value=1, description='initE', layout=Layout(width='99%'), max=100000, style=Sl…